In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: {e}")

✅ Gemini API key setup complete.


In [2]:
import os
from kaggle_secrets import UserSecretsClient
from google.adk.agents import LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.models.google_llm import Gemini
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search
from google.genai import types
from typing import List
from google.adk.runners import InMemoryRunner
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent


In [3]:
from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
# Farmers_Friend/agent.py

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.genai import types

# ✅ Declare retry_config before using Gemini
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=4,
    initial_delay=1,
    http_status_codes=[429, 503]
)

In [5]:
import os
import shutil

# Project folder name
agent_folder = "Farmers_Friend"

# ✅ Step 1: Remove if it already exists
if os.path.exists(agent_folder):
    print(f"⚠️ Folder '{agent_folder}' already exists. Deleting it to avoid ADK conflict...")
    shutil.rmtree(agent_folder)
    print("✅ Old folder deleted.")

# ✅ Step 2: Create fresh ADK agent folder
!adk create Farmers_Friend --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

print("✅ ADK agent project created successfully.")



Agent created in /kaggle/working/Farmers_Friend:
- .env
- __init__.py
- agent.py

✅ ADK agent project created successfully.


In [6]:
%%writefile Farmers_Friend/agent.py
from google.adk.agents import Agent, LlmAgent, SequentialAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools import AgentTool, FunctionTool, google_search, preload_memory,  load_memory
from google.genai import types

# Retry configuration
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=4,
    initial_delay=1,
    http_status_codes=[429, 503]
)
#  Define auto-save memory callback
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


# --- Tool Agents ---
weather_agent = Agent(
    name="weather_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a weather intelligence specialist. **Extract the sowing/harvest month from the query (e.g., "February" or "2024-03").**

**STEP 1: Determine Analysis Period**
- Identify the specific MONTH(s) from the query (e.g., "March-April 2024")
- If no month given, use TODAY's date to determine current season
- Calculate: SOWING_MONTH and HARVEST_MONTH based on crop timeline

**STEP 2: Gather Seasonal Forecast (Baseline)**

Use `google_search` for official seasonal predictions:

- "IMD long range forecast MONTH Year LOCATION"

- "Skymet seasonal outlook LOCATION MONTH"

- "IMD monsoon forecast  state wise"

Extract:
- Normal rainfall range (mm)
- Temperature deviation (°C)
- Humidity patterns
- Seasonal onset/withdrawal dates

**STEP 3: Gather Current Affairs-Based Alerts (Critical)**
Search for **real-time weather anomalies**:

- "El Niño/La Niña impact India  site:imd.gov.in"

- "IMD weather warning LOCATION today site:gov.in"

- "heat wave alert LOCATION current week"

- "flood/drought alert state site:ndma.gov.in"

- "cyclone forecast Bay of Bengal current year"

- "unseasonal rainfall alert LOCATION agriculture"


**STEP 4: Compile Alert Level**

Based on current affairs, assign risk level:

- **HIGH**: Active IMD warnings, El Niño impact, cyclone alerts

- **MEDIUM**: Below-normal monsoon forecast, heat wave trends

- **LOW**: Normal conditions, no alerts


**STEP 5: Output Strict Format**
""",
    tools=[google_search]
)
harvest_timeline_agent = LlmAgent(
    name="harvest_timeline_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Use google_search to find CROP calendar for LOCATION. Return: CROP, SOWING_WINDOW, HARVEST_WINDOW, DURATION_DAYS, SEASON_TYPE.""",
    tools=[google_search]
)
soil_agent = LlmAgent(
    name="soil_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Use `google_search` to find ** Publically available official PDFs or sources** for the soil profile of a given Indian region.
    
    Queries to try:
    - "soil health card <location> site:gov.in filetype:pdf"
    - "ICAR soil profile <district> filetype:pdf"
    
    Extract only facts like: soil type, pH, nitrogen, phosphorus, etc.
    If no data, reply: “Try https://soilhealth.dac.gov.in for full details.”
    """,
    tools=[google_search]
)
yield_agent = LlmAgent(
    name="yield_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Use `google_search` to locate publically available  yield reports published by:
    - Ministry of Agriculture
    - ICAR
    - state agri depts (e.g., krishi.maharashtra.gov.in)
    
    Sample searches:
    - "crop yield report site:data.gov.in <state> filetype:pdf"
    - "best yielding crops <state> site:gov.in"
    
    Report 2–3 crops with yields (e.g. Paddy – 4.3 t/ha). 
    Don’t guess. If search fails, say:
    “Latest yield data unavailable. Visit https://data.gov.in for updates.”
    """,
    tools=[google_search]
)
scheme_agent = LlmAgent(
    name="scheme_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Use the `google_search` tool to find government schemes (central or state-level) 
    that apply to the farmer's region and recommended crops.

    Search queries should include:
    - "<crop> subsidy <state> site:gov.in"
    - "agriculture support schemes in <district/state> for <crop>"
    - "PM-KISAN, PMFBY, soil health card scheme, MSP schemes"

    Prefer official sources like:
    - agriculture.gov.in
    - data.gov.in
    - state agri portals (e.g., krishi.maharashtra.gov.in)
    
    For each scheme, extract:
    - Name
    - Short benefit description
    - Eligibility
    - How to apply (if available)

    Return maximum 3 schemes in plain, region-aware language.
    Always cite the original source in the answer.
    """,
    tools=[google_search]
)
MultiFactorValuationAgent = LlmAgent(
    name="MultiFactorValuationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You MUST call the `google_search` tool to look up the CURRENT market price of the given product/crop/property.

    Use search phrases like:
    - "current market price of <item> in <city/state>"
    - "<item> MSP for current year site:gov.in"
    - "government subsidy scheme for <item> farming OR transport"
    - "<item> price today mandi rate"

    Your responsibilities:

    1. Fetch real-time market value of the item.
    2. Check if MSP (Minimum Support Price) exists for the item.
    3. Identify any applicable government schemes, subsidies, or incentives that reduce:
       - Production cost
       - Transportation cost
       - Storage cost
       - Insurance risk

    Consider the following factors in calculation:
       - Distance to target market (km)
       - Transport cost per km
       - Production / acquisition cost
       - Storage cost
       - Handling & packaging
       - Scheme-based subsidies or financial support
       - MSP safety net (if market price < MSP)

    Cost Adjustment Rules:
    - Reduce Total Cost by scheme/subsidy benefits.
    - If Market Price < MSP, assume MSP as fallback selling price for risk mitigation.

    Calculations:

    Adjusted Total Cost = Production Cost + Transport + Storage + Handling - Subsidies

    Effective Selling Price = 
        if MSP exists and Market Price < MSP:
            Use MSP
        else:
            Use Market Price

    Profit = Effective Selling Price - Adjusted Total Cost

    Output MUST include:
    - Market Price (real-time)
    - MSP (if applicable)
    - Schemes/Subsidies applied
    - Distance-based transport impact
    - Adjusted total cost breakdown
    - Final Profit / Loss
    - Risk Analysis (based on MSP + market trend)
    - Recommendation: Profitable / High Risk / Not Viable

    Never reply from memory — always use tool-based real-time data.
    """,
    tools=[google_search]
)
# --- Memory Loader Agent ---
memory_loader_agent = Agent(
    name="MemoryLoaderAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a memory retriever.
    Your job is to fetch any previously stored crop recommendation history for the user.
    Return the last known location, land size, budget, crop preferences, and timing.
    If no memory exists, respond with: 'No previous memory found.'
    """,
    tools=[load_memory]
)

# --- Main Crop Recommender Agent (renamed from root_agent) ---
recommendation_agent = LlmAgent(
    name="CropRecommender",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are an expert agronomist AI. Extract everything from the user's query. Do not ask for more information unless critical data is missing.

    **STEP 1: Extract from Query (Mandatory)**
    - Location (city, district, state)
    - Land size (acres)
    - Budget range (e.g., "15k-20k")
    - Farming preference: organic or conventional
    
    **STEP 2: Determine Crop Options & Harvest Timeline**
    - Use `yield_agent` with extracted state to get top 2-4 crops for that region
    - For each crop, calculate SOWING date (current date + 15 days preparation)
    - For each crop, calculate HARVEST date (sowing date + typical crop duration in days)
    - Typical durations: Paddy=120, Wheat=120, Maize=90, Sugarcane=270, Pulses=90, Oilseeds=90, Vegetables=60
    
    **STEP 3: Get Extended Weather Forecast (Critical)**
    - Use `weather_agent` for the entire period: TODAY through HARVEST DATE for each crop
    - Request: "weather forecast <location> from <today> to <harvest_date> site:imd.gov.in"
    - Get: temperature trends, rainfall, humidity, extreme weather alerts that affect entire crop cycle
    
    **STEP 4: Get Supporting Data**
    - Use `soil_agent` with extracted location for soil profile
    - Use `scheme_agent` with extracted state and each recommended crop name
    
    **STEP 5: Run Valuation for EACH Crop (Mandatory)**
    For every crop identified in Step 2, **independently call `MultiFactorValuationAgent`**:
    - Pass: crop name, location, land size, budget range
    - It will return: market price, MSP, subsidies, profit/loss, risk analysis
    
    **STEP 6: Construct Final Output (Enhanced Summary)**
    
        Format exactly:
        LOCATION: [extracted location]
        FARM PROFILE: [land_size] acres | Budget: [budget_range] | Type: [preference]
        
        
        CROP 1: [Crop Name]
      
        HARVEST TIMELINE: [from harvest_timeline_agent]
        
        WEATHER FORECAST: [from weather_agent - include full seasonal data + current affairs alerts]
        
        SOIL COMPATIBILITY: [from soil_agent]
        
        GOVERNMENT SCHEMES: [from scheme_agent - include scheme names and sources]
        
        ECONOMIC VALUATION: [from MultiFactorValuationAgent]

         
        CROP 2: [Crop Name]
        [repeat structure]

        
         STRATEGIC SUMMARY & RISK ADVISORY
    
         BEST OPTIONS:
        
        Highest Profit: [crop name] (Valuation: [profit amount])
        
        Lowest Risk: [crop name] (Weather Stability: [low/medium/high])
        
        Best Organic Fit: [crop name] (Compatibility: [soil/weather match])
        
        
         WEATHER DISRUPTION ALERTS (Current Affairs):
        
        Primary Risk: [Extract from weather_agent: El Niño/IMD Warning/Cyclone/Unseasonal Rain]
        
        Harvest Season Threat: [Specific months at risk]
        
        Risk Level: HIGH/MEDIUM/LOW
        
        Affected Crops: [list crops most vulnerable to this alert]
        
        
         RECOMMENDED PREVENTION MEASURES:
        
        For [Primary Risk]:
        
        [Actionable step 1: e.g., "Advance sowing by 15 days to avoid late monsoon"]
        
        [Actionable step 2: e.g., "Install drip irrigation for drought resilience"]
        
        [Actionable step 3: e.g., "Apply for PMFBY insurance before sowing deadline"]
        General Preparedness:
        
        Monitor IMD weekly advisories at https://www.imd.gov.in
        
        Maintain 20% budget reserve for contingency inputs
        
        
         FINAL STRATEGIC ADVICE:
        Considering [Primary Risk] alert and your [preference] farming preference, prioritize [recommended crop] with [specific prevention measure]. Budget [X%] for contingency.
        
        **=== STEP 7: Final Validation ===**
        - Verify every `[]` contains text (never None/empty)
        - Verify all three crops have complete sections
        - Verify Weather Alert section has specific current affairs data
        - Verify Prevention Measures are tailored to the alert type
        - Verify Scheme Links are extracted from scheme_agent output
        - **If any section missing → insert "Data pending - consult local KVK"**
            
    **CRITICAL RULES:**
    -  **NEVER ask user for more information only if district , state, land size or budget is not mentioned then you can 
    ask clarifying question**
    -  **Get weather data from today till the recommended crop growing season** (today → harvest), not just 2-3 months
    -  **Run valuation PER CROP** using MultiFactorValuationAgent
    -  **Include weather, soil, scheme details** for each crop
    -  **Respect budget and organic preference** in recommendations
    """,
    tools=[
        AgentTool(agent=weather_agent),
        AgentTool(agent=soil_agent),
        AgentTool(agent=yield_agent),
        AgentTool(agent=scheme_agent),
        AgentTool(agent=MultiFactorValuationAgent),
        AgentTool(agent=harvest_timeline_agent),
        AgentTool(agent=memory_loader_agent)
    ],
    output_key="Crop_Recommendation"
)



# --- Critic & Refiner ---
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a strict and unforgiving crop recommendation validator.

INPUT: {Crop_Recommendation}

You must return one of:
- "APPROVED"
-  Exactly 2 or 3 **actionable feedback** lines for correction

Your validation checklist:

1. All these sections MUST be present (at least once for 2 crops):
   - LOCATION:
   - FARM PROFILE:
   - CROP X:
   - HARVEST TIMELINE:
   - WEATHER FORECAST:
   - SOIL COMPATIBILITY:
   - GOVERNMENT SCHEMES:
   - ECONOMIC VALUATION:
   - STRATEGIC SUMMARY & RISK ADVISORY

2. All placeholders (e.g., `[]`, `TBD`, `None`) MUST be filled. If found → REJECT.

3. Section content must be real and varied:
   - WEATHER must mention La Niña, cyclone, rainfall, or similar alerts
   - SCHEMES must include specific program names (e.g., PKVY, MOVCD-NER)
   - VALUATION must mention ₹ or MSP/profit numbers

4.  Reject if:
   - Less than 2 crops fully filled
   - Crops don’t include soil + weather + valuation + scheme
   - Summary section is missing recommended crop or prevention plan

IF ALL PASS:
Respond ONLY: `APPROVED`

ELSE:
Give 2-3 corrections like:
- "Crop 2 missing scheme details"
- "Weather alert lacks real warning from IMD"
- "Missing final strategic advice"

Do NOT explain the reasoning—only give the bullet points.

Your review drives refinement — be unforgiving.
""",
    output_key="critique"
)

def exit_loop():
    return {"status": "approved", "message": "Recommendation approved. Exiting loop."}

refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a recommendation refiner. Analyze the critique.

    Original Recommendation: {Crop_Recommendation}
    Critique: {critique}

    - IF critique EXACTLY "APPROVED", call `exit_loop` function and NOTHING else
    - OTHERWISE, rewrite the recommendation incorporating ALL feedback
    -out should follow the same format as defined in recommendation_agent
    """,
    output_key="Crop_Recommendation",
    tools=[FunctionTool(exit_loop)]
)

recommendation_refinement_loop = LoopAgent(
    name="RecommendationRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2
)



# --- Pipeline with Auto Memory Save ---
crop_recommendation_pipeline = SequentialAgent(
    name="CropRecommendationPipeline",
    sub_agents=[recommendation_agent, recommendation_refinement_loop],
    after_agent_callback=auto_save_to_memory
)

# Clone versions
critic_agent_clone = Agent(
    name="CriticAgentClone",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=critic_agent.instruction,
    output_key="critique"
)

refiner_agent_clone = Agent(
    name="RefinerAgentClone",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=refiner_agent.instruction,
    output_key="Crop_Recommendation",
    tools=[FunctionTool(exit_loop)]
)

# --- Root Agent that wraps pipeline and extra loop ---
root_agent = SequentialAgent(
    name="root_agent",
    sub_agents=[
        crop_recommendation_pipeline,
        LoopAgent(
            name="refinement_loop",
            sub_agents=[critic_agent_clone, refiner_agent_clone],
            max_iterations=2
        )
    ]
)


Overwriting Farmers_Friend/agent.py


In [7]:
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --log_level DEBUG --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [93]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http:/